In [ ]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'




In [ ]:
# %%
import tensorflow as tf

from tensorflow.keras.datasets import cifar10

from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input, Dense, Flatten, Resizing, Rescaling
from tensorflow.keras.models import Model

from tensorflow.keras.layers import LayerNormalization
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

from tensorflow.keras import backend as K
from tensorflow_addons.layers import GroupNormalization, InstanceNormalization

import matplotlib.pyplot as plt



In [ ]:
# %%
#pip install clyent==1.2.1 pyyaml==6.0.1

# %%
#!pip show tensorflow-addons
#!pip show tensorflow
#!pip install tensorflow-addons==0.22.0
#!pip install tensorflow==2.15.0


# %%
#!pip list

# %%
#pip install --upgrade pip

# %%
#pip install tensorflow-privacy==0.5.0

# %%
#pip install python3.11

# %%
#!pip install cmake

# %%
#pip install matplotlib



In [ ]:
# %%
# Load CIFAR10 dataset: 60k 32x32 RGB images (50k training set + 10k test set)
(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()

# %%
# Normalize pixel values
#train_data, test_data = train_data / 255.0, test_data / 255.0

# %%
# Normalize the pixel values of the train and test data
train_data = train_data.astype('float32') / 255.0
test_data = test_data.astype('float32') / 255.0

# %%
# ResNet50 expects input size of (224, 224, 3)
resize_and_rescale = tf.keras.Sequential([
    Resizing(224, 224),
        Rescaling(1./255)
])

In [ ]:
# %%
#10 lAYERS

model_cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),  # Layer 1: Conv2D
    LayerNormalization(),  # Layer 2: LayerNormalization
    tf.keras.layers.MaxPooling2D((2, 2)),  # Layer 3: MaxPooling2D
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),  # Layer 4: Conv2D
    tf.keras.layers.Dropout(0.25),  # Layer 5: Dropout
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),  # Layer 6: Conv2D
    GroupNormalization(),  # Layer 7: GroupNormalization
    tf.keras.layers.MaxPooling2D((2, 2)),  # Layer 8: MaxPooling2D
    tf.keras.layers.Flatten(),  # Layer 9: Flatten
    tf.keras.layers.Dense(128, activation='relu'),  # Layer 10: Dense
    tf.keras.layers.Dense(10, activation='softmax')  # Output layer: Dense with softmax for multi-class classification
])



In [ ]:
# %%
# Define the input tensor
input_tensor = Input(shape=(32, 32, 3))

# Resizing and Rescaling layers
# Here assuming that Resizing is to a larger dimension such as (224, 224)
# And Rescaling is to ensure pixel values are between 0 and 1
x = Resizing(224, 224, interpolation='bilinear')(input_tensor)
x = Rescaling(1./255)(x)

# Instantiate the ResNet50V2 model
# Note: weights=None implies random initialization since CIFAR-10 is a different domain from ImageNet
# include_top=False removes the fully connected layers at the top of the network, allowing for custom classifier layers
base_model = ResNet50V2(weights=None, include_top=False, input_tensor=x)

# Ensure that the base model's layers are set to be non-trainable
base_model.trainable = False

# Add custom layers on top of the base model
x = Flatten()(base_model.output)
output_layer = Dense(10, activation='softmax')(x)  # CIFAR10 has 10 classes






In [ ]:
# Create the final model
model = Model(inputs=input_tensor, outputs=output_layer)

In [ ]:
# %%
# resnetv2 Model summary to verify the number of layers
model.summary()


In [ ]:
# %%
# cnn Model summary to verify the number of layers
model_cnn.summary()

In [ ]:

# %%
#ε and δ: Set your ε (epsilon) and δ (delta) values.
epsilon = 3.0
delta = 1e-5

In [ ]:
# Replace with your dataset size
num_examples = len(train_data)  

# %%

# Define parameters for DP-SGD FOR RESNET50V2
noise_multiplier = 0.937  # Adjusted for tighter privacy guarantee
l2_norm_clip = 1.5
batch_size = 30
learning_rate = 0.001
epochs = 200 

In [ ]:
# %%
# Define parameters for DP-SGD FOR CNN
noise_multiplier_cnn = 3.03  # Adjusted for tighter privacy guarantee
l2_norm_clip_cnn = 1.5
batch_size_cnn = 300 
learning_rate_cnn = 0.001
epochs_cnn = 500 

In [ ]:
# %%
# Compute the privacy budget expended during ResNetV2 training
epsilon= compute_dp_sgd_privacy.compute_dp_sgd_privacy(
    len(train_data), 
    batch_size, 
    noise_multiplier,
    epochs, 
    delta=delta)[0]

# %%
print(f"Trained with DP-SGD with ε = {epsilon:.2f}")

In [ ]:
# Compute the privacy budget expended during CNN training

epsilon= compute_dp_sgd_privacy.compute_dp_sgd_privacy(
    len(train_data), 
    batch_size = batch_size_cnn, 
    noise_multiplier = noise_multiplier_cnn,
    epochs= epochs_cnn, 
    delta=delta)[0]


# %%
print(f"Trained with DP-SGD with ε = {epsilon:.2f}")

# %%

In [ ]:
# Create DP-SGD optimizer model ResNetV2
optimizers = DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=batch_size,
    learning_rate=learning_rate
)

#l2_norm_clip: This parameter sets a threshold for clipping the L2 norm of gradients, which prevents any single 
#data point from having a disproportionate impact on the computation of gradients, thereby safeguarding individual 
#data point privacy.

#noise_multiplier: This determines the amount of random noise added to the gradients during training. The noise helps 
#mask the contribution of individual data points, which is central to achieving differential privacy.

#num_microbatches: This parameter controls the subdivision of a batch into smaller units, or microbatches. Processing 
#these microbatches separately and then aggregating their gradients ensures that the influence of any single data point 
#is limited.

#learning_rate: While not directly related to privacy, the learning rate can impact the convergence of the training 
#process, especially when combined with the other DP parameters.


# %%

In [ ]:

# Create DP-SGD optimizer model cnn
optimizers_cnn = DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip_cnn,
    noise_multiplier=noise_multiplier_cnn,
    num_microbatches=batch_size_cnn,
    learning_rate=learning_rate_cnn
)

In [ ]:

# %%
#loss forresnet 
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)

#since our model is already normalized with softmax we set the logits to false
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

# %%
#loss for cnn
#loss_cnn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)

#since our model is already normalized with softmax we set the logits to false
loss_cnn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)



In [ ]:
# %%
# Define RMSE as a custom metric
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
# %%
#define model parameters for cnn
model_cnn.compile(optimizer=optimizers_cnn, loss=loss_cnn, metrics=['accuracy', rmse])



In [ ]:
# %%
#define model parameters for resnetv2
model.compile(optimizer=optimizers, loss=loss, metrics=['accuracy', rmse])

In [ ]:

# %%
# cnn Train the model on 50k training images and validate on 10k test images
# Train the model with differential privacy
history_cnn = model_cnn.fit(
    train_data, 
    train_labels, 
    epochs=epochs_cnn, 
    batch_size=batch_size_cnn, 
    validation_data=(test_data, test_labels)
    )


In [ ]:

# %%
# resnetv2 Train the model on 50k training images and validate on 10k test images
# Train the model with differential privacy
history = model.fit(
    train_data, 
    train_labels, 
    epochs=epochs, 
    batch_size=batch_size, 
    validation_data=(test_data, test_labels)
)

In [ ]:
# %%
# Resnetv2 After training, multiply accuracy by 100 to convert to percentage
final_accuracy = history.history['accuracy'][-1] * 100
final_val_accuracy = history.history['val_accuracy'][-1] * 100

In [ ]:

# %%
# cnn After training, multiply accuracy by 100 to convert to percentage
final_accuracy_cnn = history_cnn.history['accuracy'][-1] * 100
final_val_accuracy_cnn = history_cnn.history['val_accuracy'][-1] * 100

In [ ]:

# %%
print(f"Final training accuracy for ResNetV2: {final_accuracy}%")
print(f"Final validation accuracy for ResNetV2: {final_val_accuracy}%")

In [ ]:

# %%
print(f"Final training accuracy for cnn: {final_accuracy_cnn}%")
print(f"Final validation accuracy for cnn: {final_val_accuracy_cnn}%")

In [ ]:
# %%
# Assuming you have the history object from model.fit
# Example: history = model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(test_data, test_labels))
import matplotlib.pyplot as plt

In [ ]:

# Extracting metrics from the history object resnetv2
loss = history.history['loss']
val_loss = history.history['val_loss']
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
rmse = history.history['rmse']
val_rmse = history.history['val_rmse']

In [ ]:


# Extracting metrics from the history object cnn
loss_cnn = history_cnn.history['loss']
val_loss_cnn = history_cnn.history['val_loss']
accuracy_cnn = history_cnn.history['accuracy']
val_accuracy_cnn = history_cnn.history['val_accuracy']
rmse_cnn = history_cnn.history['rmse']
val_rmse_cnn = history_cnn.history['val_rmse']

In [ ]:
# Number of epochs
epochs = range(1, len(loss) + 1)

epochs_cnn = range(1, len(loss_cnn) + 1)
# ... [Your existing code] ...

In [ ]:
# Plotting Loss
plt.figure(figsize=(12, 6))
plt.subplot(2, 2, 1)
plt.plot(epochs, loss, 'b-', label='ResNetV2 Training Loss')
plt.plot(epochs, val_loss, 'r-', label='ResNetV2 Validation Loss')
plt.plot(epochs_cnn, loss_cnn, 'b--', label='CNN Training Loss')
plt.plot(epochs_cnn, val_loss_cnn, 'r--', label='CNN Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:

# Plotting Accuracy
plt.subplot(2, 2, 2)
plt.plot(epochs, accuracy, 'g-', label='ResNetV2 Training Accuracy')
plt.plot(epochs, val_accuracy, 'k-', label='ResNetV2 Validation Accuracy')
plt.plot(epochs_cnn, accuracy_cnn, 'g--', label='CNN Training Accuracy')
plt.plot(epochs_cnn, val_accuracy_cnn, 'k--', label='CNN Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:


# Plotting RMSE
plt.subplot(2, 2, 3)
plt.plot(epochs, rmse, 'y-', label='ResNetV2 Training RMSE')
plt.plot(epochs, val_rmse, 'c-', label='ResNetV2 Validation RMSE')
plt.plot(epochs_cnn, rmse_cnn, 'y--', label='CNN Training RMSE')
plt.plot(epochs_cnn, val_rmse_cnn, 'c--', label='CNN Validation RMSE')
plt.title('Training and Validation RMSE')
plt.xlabel('Epochs')
plt.ylabel('RMSE')
plt.legend()

In [ ]:
plt.tight_layout()
plt.show()
